# Overview on worldwide carbon emissions and electricity sources

#### Analysis on worldwide CO2 emissions, the impact different electricity sources have on the carbon footprint of a country and how they can help smooth sudden energy prices movements.

### 1.0 Purpose of this analysis
This analysis can be divided into two parts.
In the first part, the goal is to assess the status of ___worldwide CO2 emissions___ as of 2023, as well as the impact of ___various electricity production sources___ on a country's carbon footprint and how these trends evolved over time. Particular focus has been dedicated to CO2 emissions from combustion processes (in mln tons) rather than countries' total emissions, as the former are more relevant to electricity production processes.    
In the last sections, the information from the first part is utilized to assess how the energy mix of a country can help smooth the impact of sudden movements in energy's market prices.

### 2.0 Overview of the data  
The dataset presented in this analysis has been sourced from the Energy Institute website, and includes information on carbon emissions and electricity production from various sources, mainly fossil fuels, nuclear energy, renewables and other sources.    
The data is a panel data, providing insight on countries from all over the globe and across various years. It has been however chosen to focus on the year 2023, the latest year available, to provide a quick overview of the most recent information on the matter.   
An overview of the most relevant attributes is provided below, after selecting the features to be included in the analysis.


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import warnings
import seaborn as sns
import polars
from scipy.stats import pearsonr
from scipy.stats import ks_2samp
from sklearn.preprocessing import PowerTransformer
from utils import DownloadSave

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Download and save data
panelUrl = "https://www.energyinst.org/__data/assets/excel_doc/0007/1055752/merged_panel.xlsx"
panelFile = "C:/Users/Acer/Documents/carbon-footprint/Data/Panel Data.xlsx"

glossaryUrl = "https://www.energyinst.org/__data/assets/excel_doc/0020/1540505/Consolidated-Dataset-Narrow-Format-Glossary.xlsx"
glossaryFile = "C:/Users/Acer/Documents/carbon-footprint/Data/Glossary.xlsx"

panelDataDowloader = DownloadSave(panelUrl, panelFile)
panelData = panelDataDowloader.downloadSave()

glossaryDownloader = DownloadSave(glossaryUrl, glossaryFile)
glossaryData = glossaryDownloader.downloadSave()

panelData.head()
glossaryData.head()

,Code,Variable,Units
0,biodiesel_cons_kboed,biodiesel consumption,thousand barrels of oil equivalent per day
1,biodiesel_cons_pj,biodiesel consumption,Petajoules
2,biodiesel_prod_kboed,biodiesel production,thousand barrels of oil equivalent per day
3,biodiesel_prod_pj,biodiesel production,Petajoules
4,biofuels_cons_ej,biofuels consumption,Exajoules
